# 🔬 Data exploration and preparation
In this notebook, you will look at the data and create a subset of that data. The dataset was "relatively" clean upon download, but the lecturers got rid of some pesky delimiter issues. If you want to encounter these issues yourself, you can use the original dataset found at the [Book-Crossing Dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/).

**Install packages**

In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 1.5 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=8b9b1aa883ef45f0ca2102bbbcddd18940dae7ae821d45

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1. Loading the data
Load the three datasets and explore the data.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/ADS/Recommender Systems/INFOMPPM-main/INFOMPPM-main/Week 01/data/"

In [ ]:
rates = pd.read_csv(path + "BX-Book-Ratings.csv",sep = ";")
books = pd.read_csv(path + "BX-Books.csv",sep = ";", encoding ='Latin-1')
users = pd.read_csv(path + "BX-Users.csv",sep = ";", encoding = 'Latin-1')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### 2. Cleaning the data
Check if all reviews are connected to a book. Is there a review but no book or user connected to this review? Check if all the authors are spelled correctly, are there outliers (e.g., users with insane number of reviews) etc etc. What were your steps in cleaning the data?

In [ ]:
#merge books and rates dataset in one dataframe
book_rates = pd.merge(books, rates, 
                   on='ISBN', 
                   how='outer')

In [ ]:
book_rates_miss = book_rates[['ISBN','Book-Rating']].isna().sum()

In [ ]:
book_rates[['ISBN','Book-Rating']].isna().sum()

ISBN           0
Book-Rating    0
dtype: int64

In [ ]:
#1209 reviews are missing from books
book_rates_miss

ISBN           0
Book-Rating    0
dtype: int64

In [ ]:
#merge usersID and rates dataset in one dataframe
user_rates = pd.merge(users, rates, 
                   on='User-ID', 
                   how='outer')

In [ ]:
user_rates_miss = user_rates[['User-ID','Book-Rating']].isna().sum()

In [ ]:
#173575 reviews are missing from users
user_rates_miss

User-ID             0
Book-Rating    173575
dtype: int64

In [ ]:
book_rates.dropna(inplace= True)

In [ ]:
user_rates.dropna(inplace= True)

In [ ]:
#calculate how many book rating there are for each book 
grouped_book_rates = book_rates.groupby(['Book-Rating','ISBN'])['Book-Rating'].count().reset_index(name="count").sort_values(by= "count",ascending= False)
grouped_book_rates

,Book-Rating,ISBN,count
166938,0.0,0971880107,1921
28947,0.0,0316666343,588
5503,0.0,0060928336,412
56685,0.0,0385504209,396
69952,0.0,044023722X,366
...,...,...,...
203526,2.0,0380977761,1
203525,2.0,0380975602,1
203524,2.0,0380974037,1
203523,2.0,0380973812,1


In [101]:
#Put a condition to reduce the dimensions
book_rates_new = grouped_book_rates.loc[(grouped_book_rates['count'] >= 20) & (grouped_book_rates['count'] <= 1000)]
book_rates_new.shape


(4815, 3)

In [ ]:
#calculate how many book ratings there are from each user 
grouped_user_rates = user_rates.groupby(['Book-Rating','User-ID'])['Book-Rating'].count().reset_index(name="count").sort_values(by= "count",ascending= False)
grouped_user_rates

,Book-Rating,User-ID,count
26109,0.0,198711,7533
20310,0.0,153662,4140
10048,0.0,76352,3304
14672,0.0,110973,2807
30330,0.0,230522,2663
...,...,...,...
57889,6.0,24771,1
57888,6.0,24768,1
57887,6.0,24767,1
57883,6.0,24659,1


In [ ]:
#Put a condition to reduce the dimensions
user_rates_new = grouped_user_rates.loc[(grouped_user_rates['count'] >= 100) & (grouped_user_rates['count'] <= 3000)]
user_rates_new.shape

(1057, 3)

In [ ]:
#check if there are missing values
print("Missing values for user_rates is:" , user_rates_new.isna().sum())

Missing values for user_rates is: Book-Rating    0
User-ID        0
count          0
dtype: int64


In [ ]:
print("Missing values for user_rates is:" , book_rates_new.isna().sum())

Missing values for user_rates is: Book-Rating    0
ISBN           0
count          0
dtype: int64


### 3. Subsetting the data
The publication accompanied with this dataset [Improving Recommendation Lists Through Topic Diversification](http://www2.informatik.uni-freiburg.de/~cziegler/BX/WWW-2005-Preprint.pdf) by Cai-Nicolas Ziegler, Sean M. McNee, Joseph A. Konstan, Georg Lausen; describes the process of subsetting (condensation steps) the dataset as follows (p5): 

> Hence, we discarded all books missing taxonomic descriptions, along with all ratings referring to them. Next, we also removed book titles with fewer than 20 overall mentions. Only community members with at least five ratings each were kept. 

Explore what these parameters mean for the overall dataset. Also, consider if you want the implicit ratings (Book-Rating == 0) in the final dataset. What would the implications be? Would you exclude it before the other parameters, or would you exclude them afterwards? 

While the publication describes the resulting dataset's dimensions, your results might differ. But that is ok for now.


In [ ]:
# code goes here


### 4. Extra step
Take a closer look at `BX-Books.csv` and search for a book named _Robots and Empire_ by Isaac Asimov. What do you encounter? Is this something you would solve? 

Let us argue that this is problematic for our dataset. How would you solve this? You might want to redo step 2 if you choose to take this extra step.

In [ ]:
# code goes here
books[books['Book-Title'].str.contains('Robots and Empire')]


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
19463,0586062009,Robots and Empire,Isaac Asimov,1986,HarperCollins,http://images.amazon.com/images/P/0586062009.0...,http://images.amazon.com/images/P/0586062009.0...,http://images.amazon.com/images/P/0586062009.0...
83090,0345328949,Robots and Empire,Isaac Asimov,1991,Del Rey Books,http://images.amazon.com/images/P/0345328949.0...,http://images.amazon.com/images/P/0345328949.0...,http://images.amazon.com/images/P/0345328949.0...
136152,0385190921,Robots and Empire,Isaac Asimov,1985,Smithmark Pub,http://images.amazon.com/images/P/0385190921.0...,http://images.amazon.com/images/P/0385190921.0...,http://images.amazon.com/images/P/0385190921.0...


### 5. Save the new dataset(s)
Save the dataset(s) in distinct named CSV-files for later usage. Move the file(s) to the data directory.


In [98]:
# code goes here
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
path = "/content/drive/MyDrive/ADS/Recommender Systems/INFOMPPM-main/INFOMPPM-main/Week 01/data/"

user_rates_new.to_csv(path + "user_rates.csv",index= False)
book_rates_new.to_csv(path + "book_rates.csv", index= False )